$\begin{align}
	&u_t + c(x)u_x = 0, \text{ em } [0,5] \times [0,6.4], \\
	&u(x,0) = f(x), \\~\\
	&c(x) = 1/5+\sin(x-1)^2, \\
	&f(x) = e^{-100(x-1)^2}
\end{align}$

In [1]:
using Pkg; Pkg.activate(".")

  Activating new project at `c:\Users\gozan\.vscode\Julia\SHOPDEPINN\Poster`


In [2]:
using JLD; D = load("Upwind_Dados_grad.jld")["D"]; xt_D = D[1]; u_D = D[2];

In [3]:
using NeuralPDE, Lux, Random, Optimization, OptimizationOptimisers, ModelingToolkit, Zygote, Plots
import ModelingToolkit: Interval
import Optimization: OptimizationFunction, OptimizationProblem, solve
Random.seed!(0);

In [4]:
xmin = 0; xmax = 5; tmin = 0; tmax = 6.4
c(x) = 1/5+sin(x-1)^2
f(x) = exp(-100(x-1)^2)
aux(x,t) = ((t-2.0491*x) <= 1.4279) * ((t-2.0194*x) >= -4.9414)
u_exata(x,t) = exp(-100*(atan((1/sqrt(6))*tan(atan(sqrt(6)*tan(x-1))-(sqrt(6)/5)*t)))^2);

In [5]:
Neurons = 10
Layers = 10

N_D = length(u_D)
N_I = 10000;

In [6]:
@parameters x t
@variables u(..)
Dt = Differential(t)
Dx = Differential(x)

PDE = [Dt(u(x,t)) + c(x)*Dx(u(x,t)) ~ 0]
IC = [u(x, 0) ~ f(x)]

Ω = [x ∈ Interval(xmin, xmax), t ∈ Interval(tmin, tmax)];

In [7]:
Lux_NN = Chain(    
	Dense(2, Neurons, Lux.tanh; 
	init_weight = Lux.glorot_uniform, init_bias = Lux.glorot_uniform),

    [Dense(Neurons, Neurons, Lux.tanh; 
	init_weight = Lux.glorot_uniform, init_bias = Lux.glorot_uniform) for i in 1:1:Layers],

    Dense(Neurons, 1; 
	init_weight = Lux.glorot_uniform, init_bias = Lux.glorot_uniform))

global Weight_NN = Lux.setup(Random.default_rng(0), Lux_NN)[1];

In [8]:
Weight_NN

(layer_1 = (weight = Float32[-0.6242165 0.56866914; -0.06330246 0.34922996; … ; -0.12525141 -0.093905956; -0.66517866 -0.21002297], bias = Float32[-0.0018067957, -0.4530393, 0.015580686, -0.5657447, -0.022980606, 0.5904931, 0.50905037, -0.11385763, 0.26840803, 0.3065478]), layer_2 = (weight = Float32[-0.5390303 0.13309713 … -0.53860843 -0.11768463; 0.060665477 -0.47446844 … 0.14195466 0.5241026; … ; 0.31228718 -0.34948128 … 0.54523396 0.11909132; 0.057770226 0.4409726 … 0.54192674 -0.31188104], bias = Float32[0.2792484, -0.17788953, 0.27264956, 0.11400413, 0.39570895, -0.322335, -0.34576032, 0.57750577, 0.30966, -0.5446188]), layer_3 = (weight = Float32[0.4736508 0.49099267 … -0.40226284 -0.2854084; 0.43182105 -0.11188734 … 0.035120655 -0.20693141; … ; 0.33726773 -0.53570324 … -0.54288644 0.16469675; -0.09623078 0.34622863 … 0.16912457 -0.36864647], bias = Float32[0.004465661, -0.29033548, -0.034027662, -0.29969162, -0.5889008, -0.33802646, 0.07544662, 0.2938569, -0.6518905, -0.6506386

In [9]:
Strategy = FixedStochasticTraining(N_D, N_I) 

Discretization = PhysicsInformedNN(Lux_NN, Strategy, init_params = Weight_NN)

@named PDE_System = PDESystem(PDE, IC, Ω, [x, t], u(x, t))

Problem_NeuralPDE = symbolic_discretize(PDE_System, Discretization)

Weight_NN = Problem_NeuralPDE.flat_init_params

Strategy.sets[1] = xt_D'

pde_loss_functions = Problem_NeuralPDE.loss_functions.pde_loss_functions
bc_loss_functions = Problem_NeuralPDE.loss_functions.bc_loss_functions
Neural_Network(x,t,θ) = Problem_NeuralPDE.phi([x,t],θ)[1];

In [ ]:
function loss_fit(θ, p)
    return sum(abs2, Neural_Network(xt_D[i,1],xt_D[i,2],θ) - u_D[i] for i = 1:length(u_D))/length(u_D) 
end
function callback_fit(p, l)
	if p.iter%100 == 0
		println("iteration: ", p.iter)
		println("loss_fit: ", l)
	end
	return false
end

OptimizationFunction_fit = OptimizationFunction(loss_fit, AutoZygote())
Epoch_fit = 1500
OptimizationProblem_fit = OptimizationProblem(OptimizationFunction_fit, Weight_NN) 
result_fit = solve(OptimizationProblem_fit, ADAM(10^-3); callback = callback_fit, maxiters = Epoch_fit)
Weight_NN = result_fit.u;

In [ ]:
Weight_NN

In [ ]:
function loss_fit_PINN(θ, α)
	fit = sum(abs2, Neural_Network(xt_D[i,1],xt_D[i,2],θ) - u_D[i] for i = 1:length(u_D))/length(u_D)
	PINN = sum(map(l -> l(θ), [pde_loss_functions; bc_loss_functions]))
    return α*fit + (1-α)*PINN
end
function callback_fit_PINN(P, l)
	if P.iter%100 == 0
		α = OptimizationProblem_fit_PINN.p
		println("α: ", α)
		println("iteration: ", P.iter)
		println("loss_fit_PINN: ", l)
		println("loss_fit: ", α*loss_fit(P.u, nothing))
		println("loss_PDE: ", (1-α)*map(l_ -> l_(P.u), pde_loss_functions)[1])
		println("loss_IC: ", (1-α)*map(l_ -> l_(P.u), bc_loss_functions)[1])
	end
	return false
end

OptimizationFunction_fit_PINN = OptimizationFunction(loss_fit_PINN, AutoZygote())
α = [0.8, 0.6, 0.4, 0.2]
Epoch_fit_PINN = [1750, 1750, 1750, 1750]
for i = 1:length(α)
    global OptimizationProblem_fit_PINN = OptimizationProblem(OptimizationFunction_fit_PINN, Weight_NN, α[i])
    result_fit_PINN = solve(OptimizationProblem_fit_PINN, ADAM(10^-3); callback = callback_fit_PINN, maxiters = Epoch_fit_PINN[i])
    Weight_NN = result_fit_PINN.u
end

In [ ]:
Weight_NN

In [ ]:
function loss_PINN(θ, p)
    return sum(map(l -> l(θ), [pde_loss_functions; bc_loss_functions]))
end
function callback_PINN(p, l)
	if p.iter%100 == 0
		println("iteration: ", p.iter)
		println("loss_PINN: ", l)
		println("loss_PDE: ", map(l_ -> l_(p.u), pde_loss_functions)[1])
		println("loss_IC: ", map(l_ -> l_(p.u), bc_loss_functions)[1])
	end
	return false
end

OptimizationFunction_PINN = OptimizationFunction(loss_PINN, AutoZygote())
Epoch_PINN = 1500
OptimizationProblem_PINN = OptimizationProblem(OptimizationFunction_PINN, Weight_NN) 
result_PINN = solve(OptimizationProblem_PINN, ADAM(10^-3); callback = callback_PINN, maxiters = Epoch_PINN)
Weight_NN = result_PINN.u;

In [ ]:
Weight_NN

In [ ]:
PINN(x,t) = Neural_Network(x,t,Weight_NN);

In [ ]:
Δx = 0.01; x = xmin:Δx:xmax
Δt = Δx/1.2; t = tmin:Δt:tmax
u_PINN = [PINN(i, j) for j in t, i in x]
heatmap(x, t, u_PINN, title="PINN")